In [175]:
import pandas as pd
import numpy as np
import re

from sklearn.preprocessing import LabelEncoder

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_columns', 60);

In [176]:
# !dir

In [177]:
def boil_file(filepath):
    re_20 = r'20[0-9]{2}'
    p1 = re.compile(re_20)
    
    lett = r'.*[a-zA-Z]{2,}.*'
    p2 = re.compile(lett)

    with open(filepath) as file:
        lines = file.readlines()
        count_20 = 0
        f_metric = 0
        f_skip = 1
        boil_lines = []

        for line in lines:
            res = p1.search(line)
            if res:
#                 print(res)
#                 print(line)
                count_20 += 1
                if count_20 == 1:
                    continue
                elif count_20 == 2:
                    f_metric = 1
                    yrs = [x for x in line.split(' ') if p1.match(x)]
                    continue
                elif count_20 == 3:
                    f_metric = 0
                    break

            if f_metric == 1:

#                 skip first line after year line
                if f_skip == 1:
                    f_skip = 0
                    continue

                w_list = line.split()
                if w_list:
                    w1 = w_list[0].lower()

#                     bunch of break statements
                    if w1 == 'notes' or w1 == 'statements' or w1 == 'number':
                        break
                    elif line.lower().find('company') != -1:
                        break

#                     t_list = [x.strip() for x in line.strip().split('    ') if x != '']
#                     line = line.strip()

#                     tokenize line
                    t_list = line.split()
                    met_st = 0
                    met_end = 0
                    
#                     calculate start and end of metric tokens and merge to from complete token
#                     TODO: this modifies the space between words. might reduce score
                    for i in range(len(t_list)-1, -1, -1):
                        if p2.match(t_list[i]):
                            met_end = i
                            break
                    t_list[met_st : met_end+1] = [' '.join(t_list[met_st : met_end+1])]
                    
                    boil_lines.append(t_list)

    return yrs, boil_lines

In [178]:
from os import listdir
from os.path import isfile, join
mypath = r'HCL ML Challenge Dataset'
# onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
test = pd.read_csv('Results.csv')
onlyfiles = [f + '.txt' for f in test.Filename]

In [179]:
def fix_val(val):
    re_par = r'\(\d+[,\d]*[.\d]*\)'
    p_par = re.compile(re_par)
    
    re_nopar = r'^\d[,\d]*[.\d]*'
    p_nopar = re.compile(re_nopar)
    
    re_brace = r'^\{\d[,\d]*[.\d]*'
    p_brace = re.compile(re_brace)
    
    if p_par.match(val):
        val = val.replace('(', '-')
        val = val.replace(')', '')

        val = val.replace(',', '')

    elif p_nopar.match(val):
        val = val.replace(',', '')

    elif p_brace.match(val):
        pass

    elif val == '-':
        pass

    else:
        val = r'nan'
    
    return val

In [195]:
def get_dict(yrs, boil_lines):
    idx = -1
    ridx = 1
    f_19 = 0
    f_melt = 0
    t_metirc = ''
    t_metric_1 = ''
    t_metric_2 = ''
    t_val = ''
    t_i = 0
    i = 0 
    
    if '2019' in yrs:
        idx = yrs.index('2019')
        ridx = idx - len(yrs)
        f_19 = 1

#     print(idx, ridx)
    res = {}

    while i < (len(boil_lines)):
        print(i)

        metric = boil_lines[i][0]
        metric = metric.replace('Â$', '&#36')
        metric = metric.replace('Â¢', '&#8364')
        metric = metric.replace('Â£', '&#163')
        
        if f_19 == 0:
            val = r'nan'

        if f_19 == 1:
            try:
                val = boil_lines[i][ridx]
                
                val = fix_val(val)
                
            except IndexError:
                val = r'nan'

        
        ws = metric.split()
        
        
#       Edge case 1  Melt a few lines together
        if ws[0] == 'Creditors:' and ws[-1] != 'year':
            t_matric_1 = boil_lines[i+1][0]
            t_val_1 = boil_lines[i+1][ridx]
            t_val_1 = fix_val(t_val_1)

            t_matric_2 = boil_lines[i+2][0]
            t_val_2 = boil_lines[i+2][ridx]
            t_val_2 = fix_val(t_val_2)
            
            if t_val_1 != 'nan' and fix_val(t_matric_1) == t_val_1:
                t_matric_1 = 'nan'
            
            if t_matric_1.split()[-1].lower() == 'year':
                metric = t_metric_1 + ' ' + metric
                res[metric] = t_val_1
#                 res[boil_lines[i+1][0]] = t_val_2
                i += 2
                continue
            elif t_matric_1=:
                continue
        
#             f_melt = 1
#             t_i = i
#             t_metric = metric
#             continue
        
        if f_melt == 1:             
            if ws[-1] == 'year':
                metric = t_metric + ' ' + metric
                f_melt = 0
                
                if t_val:
                    val = t_val
                    t_val = ''
                
            else:
                t_val = val
                continue
        
                
#         Edge case 2: Shift few values up by 1 row
        if ws[0].lower() == 'current' and val =='nan':
            try:
                t_matric_1 = boil_lines[i+1][0]
                t_val_1 = boil_lines[i+1][ridx]
                t_val_1 = fix_val(t_val_1)
                
                t_matric_2 = boil_lines[i+2][0]
                t_val_2 = boil_lines[i+2][ridx]
                t_val_2 = fix_val(t_val_2)
                
                if t_val_2 != 'nan' and fix_val(t_matric_2) == t_val_2:
                    t_matric_2 = 'nan'
                
                if t_matric_1.split()[0].lower() == 'cash' and t_matric_2 == 'nan':
                    res[metric] = t_val_1
                    res[boil_lines[i+1][0]] = t_val_2
                    i += 3
                    continue
                else:
                    continue
                
            except IndexError:
                continue
                
        if val != 'nan' and fix_val(metric) == val:
            metric = 'nan'
        
        
        res[metric] = val
        i += 1
        
    return res

In [ ]:
i = 0

filenames = []
extvals = []

# with open('submission.csv', 'w') as f:
#     f.write('Filename,Extracted Values\n')
    
for file in onlyfiles:
#     if i == 10:
#         break
#     i += 1
#     print(file)
    filepath = join(mypath, file)
    yrs, boil_lines = boil_file(filepath)
    
    res = get_dict(yrs, boil_lines)
    
    filenames.append(file.split('.')[0])
    extvals.append(str(res).replace('\'', '\"'))
    
#     fin.append(file.split('.')[0] + ',' + str(res))

# with open('submission.csv', 'a') as f:
#     f.writelines(fin)

df = pd.DataFrame(columns=['Filename', 'Extracted Values'])

df['Filename'] = filenames
df['Extracted Values'] = extvals

df.to_csv('test_sub.csv', index=False)

In [ ]:
filename = r'X8XX02N7.txt'
filepath = join(mypath, filename)
yrs, boil_lines = boil_file(filepath)
res = get_dict(yrs, boil_lines)
res

In [192]:
boil_lines

[['Current Assets', '5,751', '5,751'],
 ['Net current assets (liabilities)', '5,751', '5,751'],
 ['Total assets less current liabilities', '5,751', '5,751'],
 ['Creditors: amounts falling due after more than one'],
 ['2.,531)', '(2.531)'],
 ['year'],
 ['Total net assets (liabilities)', '3,220', '3,220'],
 ['Capital and reserves', '3,220', '3,220']]